In [2]:
import numpy as np
import pandas as pd
import requests
import tqdm
import time
import random
import re

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By

from time import sleep

In [21]:
pd.set_option('display.max_columns', None)

In [4]:
url = "https://www.autocasion.com/coches-ocasion"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

session = requests.Session()

response = session.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")
print(response.status_code)

200


In [5]:
anuncios_pagina = soup.find_all("article", class_="anuncio")

total_coches = int(soup.find('div', class_="container listado").find('h1').text.strip().split('\n')[0].replace('.',''))

total_paginas = int(total_coches/len(anuncios_pagina)) + (1 if (total_coches/len(anuncios_pagina))%1 else 0)

total_paginas

6600

In [25]:
def links_coches():
    url_base = "https://www.autocasion.com"
    url_pagina = f"{url_base}/coches-ocasion?direction=desc&sort=updated_at&page="
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }
    session = requests.Session()
    
    lista_paginas = []
    anuncios_coches = []

    for pagina in range(1, total_paginas +1):
        lista_paginas.append(f'{url_pagina}{pagina}')
        
    lista_prueba = lista_paginas[:2]
        
    for pagina_url in lista_prueba:
        response = session.get(pagina_url, headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")
        anuncios = soup.find_all("article", class_="anuncio")
        
        for anuncio in anuncios:
            coche_url = anuncio.find("a")['href']
            anuncios_coches.append(f'{url_base}{coche_url}')   
    
        
    return lista_paginas, anuncios_coches

In [76]:
lista_paginas,anuncios_coches = links_coches()
anuncios_coches

['https://www.autocasion.com/coches-segunda-mano/bmw-x5-ocasion/x5-xdrive-30d-ref14681491',
 'https://www.autocasion.com/coches-km0/km-0/ford-kuga-km0/2-5-duratec-phev-178kw-auto-st-line-x-166f17ffad168f-ref14388978',
 'https://www.autocasion.com/coches-segunda-mano/ford-focus-ocasion/focus-sportbreak-1-5ecoblue-trend-1-ref14681487',
 'https://www.autocasion.com/coches-segunda-mano/ford-focus-ocasion/focus-1-0-ecoboost-mhev-trend-125-ref14681486',
 'https://www.autocasion.com/coches-segunda-mano/fiat-500-ocasion/500-1-2-pop-ref14681485',
 'https://www.autocasion.com/coches-segunda-mano/hyundai-i10-ocasion/i10-1-0-go-ref14323728',
 'https://www.autocasion.com/coches-km0/km-0/ferrari-296-gtb-km0/296-gtb-ref13260224',
 'https://www.autocasion.com/coches-segunda-mano/ferrari-portofino-ocasion/portofino-ref13271865',
 'https://www.autocasion.com/coches-km0/km-0/ferrari-296-gtb-km0/296-gtb-ref14000633',
 'https://www.autocasion.com/coches-segunda-mano/ds-ds7-crossback-ocasion/ds7-crossback-2

In [29]:
headers = {
           "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
            }
session = requests.Session()

In [31]:
# Función para buscar y extraer un valor específico de cada cadena
def buscar_detalle(texto, campo):
    # Usamos expresiones regulares para capturar el campo deseado seguido de su valor
    pattern = rf"{campo}\s*:\s*([^\n]+)"
    match = re.search(pattern, texto)
    return match.group(1).strip() if match else None

In [78]:
coches_df = dict()

for coche in tqdm.tqdm(anuncios_coches):
    print(coche)
    url_coche = coche
    response = session.get(url_coche, headers=headers)
    
    #Verificamos que la respuesta es exitosa
    if response.status_code != 200:
        print(f"Error al acceder a {url_coche}, Código de estado: {response.status_code}")
        continue
    
    soup = BeautifulSoup(response.content, "html.parser")
    info_coche = {}
        
    info_coche['provincia'] = soup.find_all('div', class_= 'etiquetas')[1].find_all('span')[-1].text
    info_coche['marca_modelo'] = soup.find('h2').text
    info_coche['fecha_matriculacion'] = soup.find('ul', class_= 'datos-basicos-ficha').find('span', class_= 'icon icon-calendario').find_previous().text.strip()
    info_coche['kilometraje'] = soup.find('ul', class_= 'datos-basicos-ficha').find('span', class_= 'icon icon-velocimetro').find_previous().text.replace('.','').replace('km','').strip()
    
    lista_detalles_vendedor = soup.find('div', class_= 'datos-concesionario').find('p').text.strip().split('\n')
    info_coche['nombre_vendedor'] = lista_detalles_vendedor[0]
    info_coche['tipo_vendedor'] = lista_detalles_vendedor[1].strip()
    try:
        #puede haber o no
        info_coche['distintivo_ambiental'] = soup.find_all('div', class_= 'etiquetas')[1].find('img')['alt']
        
    except:
        info_coche['distintivo_ambiental'] = 'no disponible'
    try:
        #puede haber o no
        info_coche['garantia'] = soup.find('ul', class_= 'datos-basicos-ficha').find('span', class_= 'icon icon-file').find_previous().text.replace('Garantía: ', '').strip()
        
    except:
        info_coche['garantia'] = 'sin garantía'
    
    
    detalles_precio = soup.find('ul', class_= 'tabla-precio').find_all('li')

    for detalle in detalles_precio:
        detalle = detalle.text.replace('\n','').strip()

        if 'Precio al contado' in detalle: 
            info_coche['precio_contado'] = detalle.replace('\n', '').split(':')[1].replace('.', '').replace('€','').strip()
        elif 'Precio de venta nuevo' in detalle: 
            info_coche['precio_nuevo'] = detalle.replace('\n', '').split('*')[1].strip().replace('.','').split()[0]
            
    if 'precio_nuevo' not in info_coche:
        info_coche['precio_nuevo'] = 'no disponible'
    
    div_ficha_tecnica = soup.find('div', class_='bloque ficha-tecnica') 
    div_no_eurotax = soup.find('div', class_= 'bloque no-eurotax')
    
    if div_ficha_tecnica:
        
        detalles_carroceria = div_ficha_tecnica.find('ul', class_= 'tab-spec-1 active').find_all('li')
        for detalle in detalles_carroceria:
            detalle = detalle.text.replace('Medidas','').strip()
            
            if 'Largo' in detalle: 
                info_coche['largo'] = detalle
            elif 'Ancho' in detalle: 
                info_coche['ancho'] = detalle
            elif 'Alto' in detalle: 
                info_coche['alto'] = detalle
            #el primer número es la capacidad del maletero normal, el segundo la capacidad del maletero con los asientos abatidos
            elif 'Capacidad del maletero' in detalle: 
                info_coche['capacidad_maletero'] = detalle
            elif 'Carrocería' in detalle: 
                info_coche['carrocería'] = detalle
            elif 'Número de plazas' in detalle: 
                info_coche['num_plazas'] = detalle
            elif 'Número de puertas' in detalle: 
                info_coche['num_puertas'] = detalle
            elif 'Carrocería' in detalle: 
                info_coche['carrocería'] = detalle
            elif 'Batalla' in detalle: 
                info_coche['batalla'] = detalle
            elif 'Carrocería' in detalle: 
                info_coche['carrocería'] = detalle
            #supongo que si hay dos pesos el segundo será la masa max. autorizada
            elif 'Peso' in detalle: 
                info_coche['peso'] = detalle
           
        detalles_consumos = div_ficha_tecnica.find('ul', class_= 'tab-spec-2').find_all('li')
        for detalle in detalles_consumos:
            detalle = ' '.join(detalle.text.replace('\n','').split())
            
            if 'Consumo medio' in detalle: 
                info_coche['consumo_medio'] = detalle
            elif 'Consumo en carretera' in detalle: 
                info_coche['consumo_carretera'] = detalle
            elif 'Consumo urbano' in detalle: 
                info_coche['consumo_urbano'] = detalle
            elif 'CO2' in detalle: 
                info_coche['CO2'] = detalle
            elif 'Depósito' in detalle: 
                info_coche['deposito'] = detalle

        detalles_motor = div_ficha_tecnica.find('ul', class_= 'tab-spec-3').find_all('li')
        for detalle in detalles_motor:
            detalle = detalle.text.replace('\n', ' ').strip()
            
            if 'Combustible' in detalle: 
                info_coche['combustible'] = detalle
            elif 'Nº de cilindros' in detalle: 
                info_coche['cilindros'] = detalle
            elif 'Cilindrada' in detalle: 
                info_coche['cilindrada'] = detalle
            elif 'Sobrealimentación' in detalle: 
                info_coche['sobrealimentacion'] = detalle

        detalles_transmision = div_ficha_tecnica.find('ul', class_= 'tab-spec-4').find_all('li')
        for detalle in detalles_transmision:
            detalle = detalle.text.replace('\n', ' ').strip()
            
            if 'Tipo de transmisión' in detalle: 
                info_coche['tipo_cambio'] = detalle
            elif 'Tracción' in detalle: 
                info_coche['traccion'] = detalle
            elif 'Número de marchas' in detalle: 
                info_coche['num_marchas'] = detalle

        detalles_prestaciones = div_ficha_tecnica.find('ul', class_= 'tab-spec-5').find_all('li')
        for detalle in detalles_prestaciones:
            detalle = detalle.text
            
            if 'Potencia (KW)' in detalle: 
                info_coche['potencia_kw'] = detalle
            elif 'Potencia (CV)' in detalle: 
                info_coche['potencia_cv'] = detalle
            elif 'Par' in detalle: 
                info_coche['par'] = detalle
            elif 'Velocidad' in detalle: 
                info_coche['velocidad_max'] = detalle
            elif 'Aceleración' in detalle: 
                info_coche['aceleracion'] = detalle
                
    elif div_no_eurotax: 
        
        ficha_no_eurotax = div_no_eurotax.find_all('ul')
        lista_ficha_no_eurotax = []
        for elemento in range(0,len(ficha_no_eurotax)):
            detalle = div_no_eurotax.find_all('ul')[elemento].text
    
            if "Carrocería" in detalle:
                info_coche['carroceria'] = buscar_detalle(detalle, "Carrocería")
                info_coche['num_puertas'] = buscar_detalle(detalle, "Número de puertas")
                info_coche['num_plazas'] = buscar_detalle(detalle, "Número de plazas")
                info_coche['peso'] = buscar_detalle(detalle, "Peso")

            elif "Consumo y emisiones" in detalle: 
                info_coche['consumo_medio']= buscar_detalle(detalle, "Medio \(l/100 km\)")
                info_coche['consumo_carretera']= buscar_detalle(detalle, "Carretera \(l/100 km\)")
                info_coche['consumo_urbano']= buscar_detalle(detalle, "Urbano \(l/100 km\)")

            elif "Motor" in detalle:
                info_coche['combustible'] = buscar_detalle(detalle, "Combustible")
                info_coche['potencia_cv'] = buscar_detalle(detalle, "Potencia")
                info_coche['cilindrada'] = buscar_detalle(detalle, "Cilindrada")
            elif "Transmisión" in detalle:
                info_coche['tipo_cambio'] = buscar_detalle(detalle, "Tipo de cambio")
                info_coche['num_marchas'] = buscar_detalle(detalle, "Marchas")
                
    coches_df[(len(coches_df))] = info_coche
    
    time.sleep(random.uniform(1, 3))


  0%|          | 0/32 [00:00<?, ?it/s]

https://www.autocasion.com/coches-segunda-mano/bmw-x5-ocasion/x5-xdrive-30d-ref14681491


  3%|▎         | 1/32 [00:03<01:37,  3.13s/it]

https://www.autocasion.com/coches-km0/km-0/ford-kuga-km0/2-5-duratec-phev-178kw-auto-st-line-x-166f17ffad168f-ref14388978


  6%|▋         | 2/32 [00:04<01:10,  2.37s/it]

...

 94%|█████████▍| 30/32 [01:10<00:04,  2.31s/it]

https://www.autocasion.com/coches-segunda-mano/opel-corsa-ocasion/1-5d-dt-74kw-100cv-gs-1665fecc71434b-ref13310954


 97%|█████████▋| 31/32 [01:13<00:02,  2.40s/it]

https://www.autocasion.com/coches-km0/km-0/dfsk-ix5-km0/ix5-1-5t-classic-4x2-aut-ref12308099


100%|██████████| 32/32 [01:15<00:00,  2.35s/it]


In [84]:
df = pd.DataFrame.from_dict(coches_df).T

df.iloc[30]

provincia                                   Vizcaya
marca_modelo            Detalles OPEL Corsa (2023) 
fecha_matriculacion            Matriculado: 03/2023
kilometraje                                   22225
nombre_vendedor                       MEURI OCASION
tipo_vendedor                           Profesional
distintivo_ambiental                  no disponible
garantia                                   24 meses
precio_contado                                16875
precio_nuevo                          no disponible
largo                                           NaN
ancho                                           NaN
alto                                            NaN
capacidad_maletero                              NaN
carrocería                                      NaN
num_plazas                                     None
batalla                                         NaN
peso                                           None
num_puertas                               5 Puertas
consumo_medi

In [86]:
df

,provincia,marca_modelo,fecha_matriculacion,kilometraje,nombre_vendedor,tipo_vendedor,distintivo_ambiental,garantia,precio_contado,precio_nuevo,largo,ancho,alto,capacidad_maletero,carrocería,num_plazas,batalla,peso,num_puertas,consumo_medio,consumo_carretera,consumo_urbano,CO2,deposito,combustible,cilindros,cilindrada,sobrealimentacion,tipo_cambio,traccion,num_marchas,potencia_kw,potencia_cv,par,velocidad_max,aceleracion,carroceria
0,Málaga,Detalles BMW X5 (2008),Matriculado: 02/2008,336200,AUTOS VILLA ROSA,Profesional,B,12 meses,10000,60800,4854 mm Largo,1933 mm Ancho,1776 mm Alto,Capacidad del maletero (l)620/1.750,CarroceríaTodo Terreno,Número de plazas5,Batalla (mm)\n2.933,Peso/Masa max. autorizado (kg)2.755,Número de puertas5,"Consumo medio 8,20 litros/100 km","Consumo en carretera 7,00 litros/100 km","Consumo urbano 10,40 litros/100 km",CO2 (g/km) 217 g/km,"Depósito de combustible (l) 85,00 litros",Combustible Diesel,Nº de cilindros 6 cilindros,Cilindrada (cm3) 2.993,Sobrealimentación Turbo,Tipo de transmisión Automática secuencial,TracciónTodo terreno tracción total permanente,Número de marchas6,Potencia (KW)173,Potencia (CV)235,Par (Nm)520,Velocidad máxima (km/h)210,"Aceleración 0-100 km/h (s)8,10",NaN
1,Álava,Detalles FORD Kuga (2024),Matriculado: 06/2024,1900,AGROMOTOR OCASION,Profesional,ECO,24 meses,46900,no disponible,NaN,NaN,NaN,NaN,NaN,None,NaN,None,5 Puertas,"0,90",None,None,NaN,NaN,Gasolina y corriente eléctrica,NaN,2.488,NaN,Automático,NaN,None,NaN,243,NaN,NaN,NaN,"4x4, SUV o pickup"
2,Madrid,Detalles FORD Focus (2020),Matriculado: 06/2020,103335,ALD ARGANDA DURSAN,Profesional,C,12 meses,12999,27586,4668 mm Largo,1825 mm Ancho,1469 mm Alto,Capacidad del maletero (l)541/1.653,CarroceríaStationwagon,Número de plazas5,Batalla (mm)\n2.700,Peso/Masa max. autorizado (kg)1.900,Número de puertas5,"Consumo medio 3,70 litros/100 km","Consumo en carretera 3,40 litros/100 km","Consumo urbano 4,30 litros/100 km",CO2 (g/km) 96 g/km,"Depósito de combustible (l) 47,00 litros",Combustible Diesel,Nº de cilindros 4 cilindros,Cilindrada (cm3) 1.498,Sobrealimentación Turbo,Tipo de transmisión Manual,TracciónTracción delantera,Número de marchas6,Potencia (KW)88,Potencia (CV)120,Par (Nm)270,Velocidad máxima (km/h)-,"Aceleración 0-100 km/h (s)10,70",NaN
3,Madrid,Detalles FORD Focus (2021),Matriculado: 07/2021,35775,ALD ARGANDA DURSAN,Profesional,ECO,12 meses,18200,25586,4378 mm Largo,1825 mm Ancho,1452 mm Alto,Capacidad del maletero (l)375/1.354,CarroceríaBerlina,Número de plazas5,Batalla (mm)\n2.700,Peso/Masa max. autorizado (kg)1.900,Número de puertas5,"Consumo medio 4,40 litros/100 km","Consumo en carretera 3,70 litros/100 km","Consumo urbano 5,10 litros/100 km",CO2 (g/km) 94 g/km,"Depósito de combustible (l) 52,00 litros",Combustible Gasolina,Nº de cilindros 3 cilindros,Cilindrada (cm3) 999,Sobrealimentación Turbo,Tipo de transmisión Manual,TracciónTracción delantera,Número de marchas6,Potencia (KW)92,Potencia (CV)125,Par (Nm)210,Velocidad máxima (km/h)-,"Aceleración 0-100 km/h (s)10,30",NaN
4,Madrid,Detalles FIAT 500 (2020),Matriculado: 07/2020,43239,ALD ARGANDA DURSAN,Profesional,C,12 meses,10800,12900,3546 mm Largo,1627 mm Ancho,1488 mm Alto,Capacidad del maletero (l)185/0,CarroceríaBerlina,Número de plazas4,Batalla (mm)\n2.300,Peso/Masa max. autorizado (kg)1.305,Número de puertas3,"Consumo medio 5,10 litros/100 km","Consumo en carretera 4,30 litros/100 km","Consumo urbano 6,40 litros/100 km",CO2 (g/km) 117 g/km,"Depósito de combustible (l) 35,00 litros",Combustible Gasolina,Nº de cilindros 4 cilindros,Cilindrada (cm3) 1.242,Sobrealimentación -,Tipo de transmisión Manual,TracciónTracción delantera,Número de marchas5,Potencia (KW)51,Potencia (CV)69,Par (Nm)102,Velocidad máxima (km/h)160,"Aceleración 0-100 km/h (s)12,90",NaN
5,Barcelona,Detalles HYUNDAI i10 (2018),Matriculado: 12/2018,33069,AUTOHERO BARCELONA,Profesional,C,12 meses,9399,13115,3665 mm Largo,1660 mm Ancho,1500 mm Alto,Capacidad del maletero (l)252/0,CarroceríaBer

## Pruebas fallos

In [64]:
url = 'https://www.autocasion.com/coches-segunda-mano/mercedes-benz-clase-slk-ocasion/slk-200-be-ref14025210'
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

session = requests.Session()

response = session.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")
print(response.status_code)

200
